In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import exp
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler


import quandl

In [ ]:
df = quandl.get("USTREASURY/YIELD")

In [ ]:
#################
def clean_data(df):
    df = df[["2 YR", "10 YR", "20 YR"]]
    df = df.dropna()
    return df

def normalize_data(df):
    scaler = MinMaxScaler()
    for column in df.columns:
        scaler.fit(df[column].values.reshape(-1,1))
        df[column] = scaler.transform(df[column].values.reshape(-1,1))
    return df

In [ ]:
df = clean_data(df)
df = normalize_data(df)

In [ ]:
df["B(0)"] = None
df["B(1)"] = None
df["B(2)"] = None

In [ ]:
import scipy as sp
from scipy import linalg

In [ ]:
def get_params(lamb,t):
    a = 1
    b = (1 -(exp(-t*lamb)))/ (t * lamb)
    c = b - exp(-t * lamb)
    return a, b, c

In [ ]:
def update_params(df):
    
    lamb = 0.0609 # Diebold-Li
    
    a2, b2, c2 = get_params(lamb, 2)
    a10, b10, c10 = get_params(lamb, 10)
    a20, b20, c20 = get_params(lamb, 20)
    
    for row in range(len(df)):
        y2 = df["2 YR"].iloc[row]
        y10 = df["10 YR"].iloc[row]
        y20 = df["20 YR"].iloc[row]
        
        x2 = [a2, b2, c2]
        x10 = [a10, b10, c10]
        x20 = [a20, b20, c20]
        
        X = [x2, x10, x20]
        X = sm.add_constant(X)
        
        y = [y2, y10, y20]
        
        model = sm.OLS(y, X).fit()
        
        df["B(0)"].iloc[row], df["B(1)"].iloc[row], df["B(2)"].iloc[row] = model.params

        
    return df

In [ ]:
update_params(df)

In [ ]:
plt.figure(figsize = (26,11))
plt.plot(df["B(0)"], color = "r", label = "beta_0")
plt.plot(df["B(1)"], color = "g", label = "beta_1")
plt.plot(df["B(2)"], color = "b", label = "beta_2")
plt.legend(loc = 4)
plt.show()

In [ ]:
import tensorflow as tf

from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Activation, Dropout, RepeatVector, Concatenate, Reshape, Input, Dot
from keras import regularizers

from keras.optimizers import SGD

from tensorflow.nn import tanh, softmax

from sklearn.metrics import mean_squared_error

In [ ]:
data = np.array(df.values[:,])

In [ ]:
train = data[:4000]
test = data[4000:]

In [ ]:
def softmax_activation(x):
    e = K.exp(x - K.max(x, axis=1, keepdims=True))
    s = K.sum(e, axis=1, keepdims=True)
    return e / s

In [ ]:
class LSTM_Attention(object):
    
    def __init__(self, X, Y, n_h1, n_h2,  attn_neurons, epochs,
                 batch_size ,
                 predict ,
                 lookback,
                 attn_sharing = True,
                 regularization = (0.00001, "0.00001"),
                 ):
        
        keras.clear_session()
        tf.reset_default_graph()
        self.learning(True)
        
                
        self.epochs = epochs
        self.batch_size = batch_size
        self.attn_sharing = attn_sharing
        
        self.X = X
        self.Y = Y
        self.n_h1 = n_h1
        self.n_h2 = n_h2
        #self.n_h3 = n_h3
        self.attn_neurons = attn_neurons
        
        self.n_obs = self.X.shape[0]
        self.len_input = self.X.shape[1]
        self.dim_input = self.X.shape[2]
        self.n_output = self.y.shape[1]
        
        self.regularization = regularization[0]
        
        
        assert self.n_input == self.n_output
        
        self.attn_activation = "selu"
        self.attn_init = "lecun_normal"
        
    def shared_layers(self):
        if self.regularization > 0:
            self.kernel_regularizer = regularizers.l2(self.regularization)
            self.LSTM_regularizer = regularizers.l2(self.regularization)
            self.dropout = 0.2
            self.bias_regularizer = regularizers.l2(self.regularizers)
        else:
            self.kernel_regularizer = self.bias_regularization = self.LSTM_regularization = None
            self.dropout = 0.2
            
        if self.attn_sharing:
            if self.attn_neurons > 0:
                self.attn_h01 = Dense(self.attn_neurons,
                                     kernel_regularizer = self.kernel_regularizer,
                                     activation = self.attn_activation,
                                     kernel_initializer = self.attn_init)
            
            self.attn_h1 = Dense(1,
                                     kernel_regularizer = self.kernel_regularizer,
                                     activation = self.attn_activation,
                                     kernel_initializer = self.attn_init)
            
        self.h2 = LSTM(self.n_h2,
                      kernel_regularizer = self.kernel_regularizer,
                                     bias_regularizer = self.bias_reularizer,
                       recurrent_regularizer = self.LSTM_regularizer,
                       recurrent_dropout = self.dropout,
                       return_state = True 
                                     )
        
        self.Output = Dense( 1,
                            kernel_regularizer = self.kernel_regularizer,
                                     bias_regularizer = self.bias_reularizer,
                            activation = "linear"
                           )
        
        def LSTMAttention(self):
            self.shared_layers()
            
            inputs = Input(shape = (self.len_input, self.dim_input))
             
            X = LSTM(self.n_h1,
                     kernel_regularizer = self.kernel_regularizer,
                                     bias_regularizer = self.bias_reularizer,
                       recurrent_regularizer = self.LSTM_regularizer,
                       recurrent_dropout = self.dropout,
                       return_sequences = True 
                    )(inputs)
            
            X = Reshape((self.len_input , self.n_h2))(X)
            
            h_t = Input(shape = (self.n_h2))
            c_t = Input(shape = (self.n_h2))
            h_t1 = h_t
            c_t1 = c_t
            
            preds = list()
            
            for i in range(self.n_outputs + 1):
                h_t1_repeat = RepeatVector(self.len_input)(h_t1)
                joined = Concatenate(axis = -1)([X, h_t1_repeat])
                
                if self.attn_neurons>0:
                    if self.attn_sharing:
                        joined = self.attn_h01(joined)
                    else:
                        joined = Dense(self.attn_neurons,
                                      kernel_regularizer = self.kernel_regularizer,
                                     activation = self.attn_activation,
                                     kernel_initializer = self.attn_init)(joined)
                
                if self.attn_sharing:
                    e_vals = self.attn_h1(joined)
                else:
                    e_vals = Dense( 1,
                                   kernel_regularizer = self.kernel_regularizer,
                                     activation = self.attn_activation,
                                     kernel_initializer = self.attn_init
                                  )(joined)
                
                
                alphas = Activation(softmax_activation)(e_vals)
                attns = Dot(axes = 1)([alphas , X])
                
                h_t1 , _, c_t1 = self.h2(attns, initial_state = [h_t1, c_t1])
                
                if i>0:
                    pred = self.Output(h_t1)
                    preds.append(pred)
            
            self.model = Model(inputs = [inputs , h_t, c_t], outputs = preds)
            self.model.compile(loss = "mse", optimizer ="adam", metrics =["mse"])
            
            print(model.summary())
            
        def fit(self):
            self.learning(True)
            
            h_t = np.zeros((self.n_obs , self.n_h2))
            c_t = np.zeros((self.n_obs, self.n_h2))
            
            y_split = np.split(self.Y, indices_or_sections = self.n_outputs, axis = 1)
        
            self.model.fit([self.X, h_t, c_t],
                          y_split,
                          epochs = self.epochs,
                          batch_size = self.batch_size,
                          shuffle = True,
                          verbose = 2,
                          validation_split = 0.3)
        

In [ ]:
N_EPOCHS = 10
N_Attention_shared = 0

# n_input = 1
# n_output = 1

n_h1 = 20
n_h2 = 20

batch_size = 32
lookback = 30
predict = 1

In [ ]:
attention = LSTM_Attention(X, Y, n_h1, n_h2,  N_Attention_shared, N_EPOCHS,
                 batch_size ,
                 predict ,
                 lookback)

In [ ]:
# class LSTM_Attention(tf.keras.Model):
#     def __init__( self , n_input, n_output, n_hidden,  batch_size,  time_step):
        
#         super(LSTM_Attention, self).__init__()
#         self.n_input = n_input
#         self.n_output = n_hidden
#         self.neurons = n_hidden
#         self.batch_size = batch_size
#         self.time_step = time_step
        
#         self.lstm_layer = LSTM(self.neurons,
#                               return_sequences = True,
#                               return_state = True,
#                               stateful = True,
#                               recurrent_initializer = "glorot_uniform")
        
#         self.Wh = Dense(self.neurons)
#         self.Ws = Dense(self.neurons)
#         self.Wx = Dense(1)
#         self.V = Dense(1)
#         self.O = Dense(self.n_output)

#     def get(self, X, hidden, s_t):
#         hidden = tf.expand_dims(hidden, 1)
#         X = tf.expand_dims(X, 0)
#         atten = self.V(tanh( self.Wx(X) + 
#                                   self.Wh(hidden) + 
#                                  self.Ws(s_t)))
#         weights = softmax(atten, axis = 1)
#         o1, s_h, s_c = self.lstm(X * weights)
#         output = self.O(s_h)
        
#         return weights, s_h, s_c, output
    
#     def zero_init(self):
#         hidden = tf.zeros((self.batch_size, self.neurons))
#         s_t = tf.zeros((self.batch_size, self.time_step, self.neurons))
        
#         return hidden, s_t
        

In [ ]:
# tf_train = tf.data.Dataset.from_tensor_slices(train_data[-250:]).window(size=time_step+shift, shift=shift, drop_remainder=True).flat_map(
#         lambda x: x.batch(time_step+shift))

# tdata = tf_train.map(lambda x: tf.reshape(x[:time_step],[time_step,1]))
# labels = tf_train.map(lambda x: x[-shift:])

In [ ]:
# def train(inp, target, hidden,s_t):
#     loss = 0
#     pred = 0
#     with tf.GradientTape() as tape:
#         for i,t in zip(inp,target):
#             weight, hidden, s_t, output = attention(tf.cast(i, tf.float32), hidden, s_t)
#             loss += tf.keras.losses.mean_squared_error(t, output)
#             pred += 1
#     variables = attention.trainable_variables
#     grads = tape.gradient(loss,variables)
#     optimizer.apply_gradients(zip(grads, variables))
#     return loss/pred

In [ ]:
# Losses = []
# for epoch in range(N_EPOCHS):
#     hidden, s_t = attention.zero_init()
#     loss = train()
    
#     Losses.append(loss)
    
#     if epoch % 2 == 0:
#         print("Epoch : {}, Loss : {}".format(epoch+1, loss.numpy()[0]))